# Build pipeline with command_component decorated python function

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define `CommandComponent` using python function and `command_component` decorator
- Create `Pipeline` using component defined by `command_component`

**Motivations** - This notebook explains how to define `CommandComponent` via Python function and `@command_component` decorator, then use command component to build pipeline.  

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [9]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: Shared token cache unavailable
	VisualStudioCodeCredential: Azure Active Directory error '(invalid_grant) AADSTS700082: The refresh token has expired due to inactivity. The token was issued on 2022-01-12T00:05:41.8429245Z and was inactive for 90.00:00:00.
Trace ID: 8d7734b2-9e4b-48d0-aaff-1ca162111700
Correlation ID: 7f897184-e9e7-4a85-92b9-c69ddcfb8573
Timestamp: 2022-10-20 00:08:05Z'
Content: {"error":"invalid_grant","error_description":"AADSTS700082: The refresh token has expired due to inactivity. The token was 

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

Found the config file in: D:\azureml-examples\config.json
Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourceGroups/azureml-examples-v2/providers/Microsoft.MachineLearningServices/workspaces/main/computes/cpu-cluster', 'Resource__source_path': None, 'base_path': 'd:\\azureml-examples\\sdk\\python\\jobs\\pipelines\\1b_pipeline_with_python_function_components', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x00000250019F7940>, 'resource_id': None, 'location': 'eastus', 'size': 'STANDARD_D2_V2', 'min_instances': 0, 'max_instances': 40, 'idle_time_before_scale_down': 1800.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x00000250019F7700>, 'tier': 'dedicated', 'subnet': None})


# 2. Import components that are defined with python function

We defined three sample components using `command_component` decorator in [components.py](src/components.py).

In [4]:
with open("src/components.py") as fin:
    print(fin.read())

from pathlib import Path
from random import randint
from uuid import uuid4

# mldesigner package contains the command_component which can be used to define component from a python function
from mldesigner import command_component, Input, Output


@command_component()
def train_model(
    training_data: Input(type="uri_file"),
    max_epochs: int,
    model_output: Output(type="uri_folder"),
    learning_rate=0.02,
):
    """A dummy train component.

    Args:
        training_data: a file contains training data
        max_epochs: max epochs
        learning_rate: learning rate
        model_output: target folder to save model output
    """

    lines = [
        f"Training data path: {training_data}",
        f"Max epochs: {max_epochs}",
        f"Learning rate: {learning_rate}",
        f"Model output path: {model_output}",
    ]

    for line in lines:
        print(line)

    # Do the train and save the trained model as a file into the output folder.
    # Here only output a dummy

**Note**: you need to install `mldesigner` package to use command_component decorator. 

`mldesigner` is a python SDK package designed to ease the authoring experience of components and pipelines.

In [5]:
# Option 1: install directly
# %pip install mldesigner

# Option 2: install as an extra dependency of azure-ai-ml
# %pip install azure-ai-ml[designer]

In [6]:
%load_ext autoreload
%autoreload 2
# import the components as functions
from src.components import train_model, score_data, eval_model

help(train_model)

Help on function train_model in module src.components:

train_model(training_data: <mldesigner._input_output.Input object at 0x0000025001B1B0A0>, max_epochs: int, model_output: <mldesigner._input_output.Output object at 0x0000025001B1B160>, learning_rate=0.02)
    A dummy train component.
    
    Args:
        training_data: a file contains training data
        max_epochs: max epochs
        learning_rate: learning rate
        model_output: target folder to save model output



You can also register component functions to workspace use `ml_client.components.create_or_update()`.

In [7]:
print(train_model)

<function train_model at 0x0000025001F68820>


# 3. Sample pipeline job

## 3.1 Build pipeline

In [17]:
cluster_name = "cpu-cluster"
# define a pipeline with component
@pipeline(default_compute=cluster_name)
def pipeline_with_python_function_components(input_data, test_data, learning_rate):
    """E2E dummy train-score-eval pipeline with components defined via python function components"""

    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    train_with_sample_data = train_model(
        training_data=input_data, max_epochs=5, learning_rate=learning_rate
    )
    score_with_sample_data = score_data(
        model_input=train_with_sample_data.outputs.model_output, test_data=test_data
    )
    # example how to change path of output on step level, please note the if the output is promote to pipeline level you need to change path in pipeline job level
    score_with_sample_data.outputs.score_output=Output(type='uri_folder', mode='rw_mount', path='azureml://datastores/workspaceblobstore/paths/${{name}}/')
    eval_with_sample_data = eval_model(
        scoring_result=score_with_sample_data.outputs.score_output
    )

    # Return: pipeline outputs
    return {
        "eval_output": eval_with_sample_data.outputs.eval_output,
        "model_output": train_with_sample_data.outputs.model_output,
    }


pipeline_job = pipeline_with_python_function_components(
    input_data=Input(
        path="wasbs://demo@dprepdata.blob.core.windows.net/Titanic.csv", type="uri_file"
    ),
    test_data=Input(
        path="wasbs://demo@dprepdata.blob.core.windows.net/Titanic.csv", type="uri_file"
    ),
    learning_rate=0.1,
)
# example how to change path of output on pipeline level
pipeline_job.outputs.model_output=Output(type='uri_folder', mode='rw_mount', path='azureml://datastores/workspaceblobstore/paths/${{name}}/')

# 3.2 Submit pipeline job

In [18]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
pipeline_samples,dynamic_honey_9plyw9cfzf,pipeline,Preparing,Link to Azure Machine Learning studio


In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)